In [1]:
import pandas as pd
import numpy as np
import datetime
import os

In [2]:
df_store_BR_list=pd.read_excel("./BL 2020Q2 Store List with QuadInfo and Q1 competitorInfo 04.27.20.xlsx",
                            dtype=str,sheet_name="Store List")
df_store_BR_list.head(2)

,Store,City,ST,Zip,Open Date,SOTF,Furniture Type,Plastic Bag Ban,Competitor Rank,Quad_2018Q2,Quad_2018Q3,Quad_2018Q4,Quad_2019Q1,Quad_2019Q2,Quad_2019Q3,Quad_2019Q4,Count_Quad_III L7Qtr,Notes
0,1,COLUMBUS,OH,43232,1983-09-15 00:00:00,Yes,FULLFURN,NO,M,Quadrant I,Quadrant IV,Quadrant I,Quadrant III,Quadrant III,Quadrant III,Quadrant III,4,NaN
1,3,AUGUSTA,GA,30906,1988-10-31 00:00:00,No,FULLFURN,NO,M,Quadrant II,Quadrant II,Quadrant I,Quadrant I,Quadrant III,Quadrant III,Quadrant III,3,NaN


In [3]:
stores_list_wanted=['420','1399','1601','1620']
df_stores_wanted=df_store_BR_list[df_store_BR_list['Store'].isin(stores_list_wanted)]
df_stores_wanted.shape

(4, 18)

In [4]:
df_stores_wanted=df_stores_wanted[['Store','City','ST','Zip','Open Date','SOTF','Furniture Type']]
df_stores_wanted['Zip'].apply(len).unique()

array([5])

In [6]:
DMA_zip=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",
                      dtype=str,skiprows=1,usecols=[0,2])
DMA_zip=DMA_zip.drop_duplicates()
DMA_zip.columns=['Zip','DMA']
DMA_zip=DMA_zip.groupby(['Zip'])['DMA'].apply(list).to_frame().reset_index()
DMA_zip=DMA_zip.rename(columns={"DMA":"DMA_list"})

In [7]:
DMA_zip.head(2)

,Zip,DMA_list
0,00501,[NEW YORK]
1,00544,[NEW YORK]


In [8]:
df_stores_wanted=pd.merge(df_stores_wanted,DMA_zip,on="Zip",how="left")
df_stores_wanted

,Store,City,ST,Zip,Open Date,SOTF,Furniture Type,DMA_list
0,420,BAY CITY,MI,48706,1992-08-03 00:00:00,NO,FULLFURN,[FLINT-SAGINAW-BAY CITY]
1,1399,SAGINAW,MI,48604,2009-10-15 00:00:00,NO,FULLFURN,[FLINT-SAGINAW-BAY CITY]
2,1601,FLINT,MI,48507,2001-06-14 00:00:00,NO,FULLFURN,[FLINT-SAGINAW-BAY CITY]
3,1620,JACKSON,TN,38305,2001-10-15 00:00:00,NO,FULLFURN,"[JACKSON, TN]"


In [9]:
# just 1 DMA per store
df_stores_wanted['DMA']=df_stores_wanted['DMA_list'].apply(lambda x: x[0])

In [10]:
TA_zips=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
TA_zips=TA_zips.parse("view_by_store",dtype=str)
TA_zips.head(2)

,TA_num,ta_name,location_id,address_line_1,address_line_2,zip_cd,city_nm,state_nm,latitude_meas,longitude_meas,DMA,CTY,all_trans_P_zips,all_trans_S_zips,zips_in_10,trans_P_zips_70_within_TA,trans_S_zips_70_within_TA
0,1,multiple_1,1949,3178 LAVON DR,NaN,75040,GARLAND,TX,32.945015,-96.619194,{'DALLAS-FT. WORTH'},{'DALLAS'},"['75040', '75044', '75048', '75098']","['75089', '75042', '75043', '75041', '75088', ...",[],"['75040', '75098', '75044', '75048']","['75088', '75042', '75082', '75043', '75089', ..."
1,1,multiple_1,1255,1418 W MOORE AVE,NaN,75160,TERRELL,TX,32.738772,-96.299877,{'DALLAS-FT. WORTH'},"{'HUNT', 'KAUFMAN', 'ROCKWALL'}","['75160', '75126', '75161']","['75142', '75169', '75474', '75143', '75114', ...",[],"['75126', '75161', '75160']","['75142', '75169', '75150', '75117', '75143', ..."


In [13]:
TA_zips=TA_zips[TA_zips['location_id'].isin(df_stores_wanted['Store'].tolist())]
TA_zips=TA_zips[['location_id','trans_P_zips_70_within_TA','trans_S_zips_70_within_TA']]
TA_zips=TA_zips.rename(columns={"location_id":"Store","trans_P_zips_70_within_TA":"P_zips","trans_S_zips_70_within_TA":"S_zips"})
TA_zips['P_zips_count']=TA_zips['P_zips'].apply(lambda x: len(eval(x)))
TA_zips['S_zips_count']=TA_zips['S_zips'].apply(lambda x: len(eval(x)))
TA_zips['TA_zips_count']=TA_zips['P_zips_count']+TA_zips['S_zips_count']
TA_zips

,Store,P_zips,S_zips,P_zips_count,S_zips_count,TA_zips_count
868,1601,"['48433', '48532', '48507', '48504', '48503', ...","['48867', '48420', '48506', '48423', '48458', ...",7,9,16
916,1620,"['38305', '38301', '38343']","['38344', '38340', '38008', '38313', '38063', ...",3,21,24
979,420,['48706'],"['48631', '48732', '48658', '48708', '48640', ...",1,11,12
985,1399,"['48706', '48609', '48603', '48604', '48601', ...","['48616', '48726', '48650', '48655', '48858', ...",8,26,34


In [14]:
df_stores_wanted=pd.merge(df_stores_wanted,TA_zips,on="Store",how="left")
df_stores_wanted

,Store,City,ST,Zip,Open Date,SOTF,Furniture Type,DMA_list,DMA,P_zips,S_zips,P_zips_count,S_zips_count,TA_zips_count
0,420,BAY CITY,MI,48706,1992-08-03 00:00:00,NO,FULLFURN,[FLINT-SAGINAW-BAY CITY],FLINT-SAGINAW-BAY CITY,['48706'],"['48631', '48732', '48658', '48708', '48640', ...",1,11,12
1,1399,SAGINAW,MI,48604,2009-10-15 00:00:00,NO,FULLFURN,[FLINT-SAGINAW-BAY CITY],FLINT-SAGINAW-BAY CITY,"['48706', '48609', '48603', '48604', '48601', ...","['48616', '48726', '48650', '48655', '48858', ...",8,26,34
2,1601,FLINT,MI,48507,2001-06-14 00:00:00,NO,FULLFURN,[FLINT-SAGINAW-BAY CITY],FLINT-SAGINAW-BAY CITY,"['48433', '48532', '48507', '48504', '48503', ...","['48867', '48420', '48506', '48423', '48458', ...",7,9,16
3,1620,JACKSON,TN,38305,2001-10-15 00:00:00,NO,FULLFURN,"[JACKSON, TN]","JACKSON, TN","['38305', '38301', '38343']","['38344', '38340', '38008', '38313', '38063', ...",3,21,24


In [15]:
writer=pd.ExcelWriter("./BL_stores_needed_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_stores_wanted.to_excel(writer,"stores_list_wanted",index=False)
writer.save()

In [17]:
df=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200401-135137-445.txt",dtype=str,sep="|")
df=df[df['location_id']=="1846"]

In [18]:
df

,location_id,location_desc,open_dt,close_dt,address_line_1,address_line_2,city_nm,state_nm,zip_cd,longitude_meas,latitude_meas,sales_square_feet,furniture_sales_square_feet
622,1846,WEST MARKET PLAZA,2004-10-22,NaN,1890 W MARKET ST,NaN,AKRON,OH,44313-6940,-000000000081.576614,000000000041.109996,00000000022156.0000,00000000002889.0000
